In [1]:
#install PyKEEN
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'pykeen'
  Cloning https://github.com/pykeen/pykeen.git to /tmp/pip-req-build-9vjvqcti
  Running command git clone -q https://github.com/pykeen/pykeen.git /tmp/pip-req-build-9vjvqcti
     |████████████████████████████████| 28.5MB 105kB/s 
     |████████████████████████████████| 307kB 37.6MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 174kB 37.8MB/s 
     |████████████████████████████████| 143kB 42.9MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 112kB 40.4MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
  Created wheel for pykeen: filename=pykeen-1.5.1.dev0-cp37-none-any.whl size=475522 sha256=7be44416ee4bbe09287d70404f0eba4d8dae8169c315e3f4d477616ae14520b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-gz6

In [2]:
#connect google drive to colab
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [1]:
#imports
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from pykeen.sampling import NegativeSampler, BasicNegativeSampler
import torch
import numpy as np

In [2]:
#paths to files
base_path = '/content/gdrive/MyDrive/datasets/kinship/'
sep = ','
dataset_name = 'kinship'
trainset_path = base_path + dataset_name + '_train.txt'
validset_path = base_path + dataset_name + '_valid.txt'
trainvalidset_path = base_path + dataset_name + '_train+valid.txt'

In [3]:
#load triples from training+validation-set file
all_triples = TriplesFactory.from_path(trainvalidset_path)

In [4]:
#load triples from training-set file and validation-set file
train_triples = TriplesFactory.from_path(trainset_path)
valid_triples = TriplesFactory.from_path(validset_path)

In [1]:
#perform a train-valid splitting
#trainset_percentage = 0.8
#train_triples, valid_triples = all_triples.split(ratios=trainset_percentage)

'\ntrainset_percentage = 0.8\ntrain_triples, valid_triples = all_triples.split(ratios=trainset_percentage)\n'

In [ ]:
#generate negative samples (if needed, e.g., for the downstream classifier)
#n_negatives_per_triple = 1
#ns_train = BasicNegativeSampler(triples_factory = train_triples, num_negs_per_pos = n_negatives_per_triple)
#negatives_train = ns_train.corrupt_batch(train_triples.mapped_triples)
#ns_validation = BasicNegativeSampler(triples_factory = validation_triples, num_negs_per_pos = n_negatives_per_triple)
#negatives_validation = ns_validation.corrupt_batch(validation_triples.mapped_triples)

In [5]:
#train embeddings
result = pipeline( #https://pykeen.readthedocs.io/en/stable/api/pykeen.pipeline.pipeline.html#pykeen.pipeline.pipeline
    training=train_triples,
    testing=valid_triples,
    #model='TransR',
    model='TransE', #https://pykeen.readthedocs.io/en/latest/reference/models.html
                    #https://pykeen.readthedocs.io/en/latest/api/pykeen.models.Model.html#pykeen.models.Model
                    #https://pykeen.readthedocs.io/en/latest/api/pykeen.models.EntityRelationEmbeddingModel.html#pykeen.models.EntityRelationEmbeddingModel
    #loss='MSELoss', #default: the default loss specific of the model to be used (in most cases, 'MarginRankingLoss'); 
                    #https://pykeen.readthedocs.io/en/stable/reference/losses.html
    #model configuration
    model_kwargs = dict(
        #relation_dim=100, #dimensionality of the relation embeddings; default: 50; 
                          #in some models (like TransE) relation_dim cannot be set: it is assumed to be equal to embedding_dim
        embedding_dim=200 #dimensionality of the netity embeddings; default: 50
    ),
    loss_kwargs = dict(), #loss configuration
    training_loop='SLCWATrainingLoop', #default: 'SLCWATrainingLoop'; https://pykeen.readthedocs.io/en/stable/reference/training.html
    regularizer='LpRegularizer', #default: 'NoRegularizer'; https://pykeen.readthedocs.io/en/stable/reference/regularizers.html
    #regularizer configuration
    regularizer_kwargs=dict(
        p=2.0, #default: 2.0
        weight=1.0 #default: 1.0
    ),
    optimizer='SGD', #default: torch.optim.Adagrad; https://pytorch.org/docs/stable/optim.html%23torch.optim
    #optimizer configuration
    optimizer_kwargs = dict(
        lr=0.01
    ),
    negative_sampler='Basic', #default: 'Basic' (pykeen.sampling.BasicNegativeSampler); only allowed with training_loop='SLCWATrainingLoop'
    #training configuration
    training_kwargs=dict(
        num_epochs=200,
        use_tqdm_batch=False,
    ),  
    #runtime configuration
    random_seed=1234,
    device='gpu'#device='cpu'
)

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...


INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=2048.


INFO:pykeen.evaluation.evaluator:Evaluation took 0.08s seconds


In [11]:
#save result to file (model and evaluation)
result_path = base_path + 'results'
result.save_to_directory(result_path)

In [6]:
#extract entity embeddings
entity_ids = torch.LongTensor(range(all_triples.num_entities))
e_emb = result.model.entity_embeddings
e_emb.cpu()
e_emb_numpy = e_emb(entity_ids).detach().numpy()
entity2embedding = {}
for eid in range(all_triples.num_entities):
  e = all_triples.entity_id_to_label[eid]
  entity2embedding[e] = list(e_emb_numpy[eid])

In [8]:
#extract relation embeddings
relation_ids = torch.LongTensor(range(all_triples.num_relations))
r_emb = result.model.relation_embeddings
r_emb.cpu()
r_emb_numpy = r_emb(relation_ids).detach().numpy()
relation2embedding = {}
for rid in range(all_triples.num_relations):
  r = all_triples.relation_id_to_label[rid]
  relation2embedding[r] = list(r_emb_numpy[rid])

In [7]:
list(entity2embedding.items())[:2]

[('person0',
  [0.0059823855,
   -0.0137435,
   -0.063965864,
   0.14920972,
   -0.046107266,
   0.069274284,
   0.060045857,
   -0.019356653,
   0.012328942,
   0.0043080975,
   -0.0986021,
   0.045453392,
   -0.02645256,
   -0.01475959,
   -0.08426816,
   -0.10231052,
   -0.042787574,
   0.11750303,
   -0.041603226,
   0.015913047,
   -0.103637375,
   -0.098502,
   -0.045422412,
   0.026805421,
   -0.019052176,
   -0.045623813,
   0.07311958,
   0.033683818,
   -0.03231793,
   0.052005805,
   -0.07769763,
   -0.105829395,
   -0.031963106,
   0.03314049,
   -0.077791415,
   -0.08164989,
   0.11940607,
   -0.060168114,
   -0.09477018,
   -0.06246809,
   0.12119943,
   -0.037389353,
   0.044556588,
   0.069831796,
   0.026076421,
   -0.11039273,
   -0.008963521,
   0.07475549,
   -0.035695355,
   0.023279455,
   0.06451965,
   0.072300404,
   -0.05732815,
   -0.032191552,
   0.023532124,
   0.039529167,
   0.07354148,
   -0.035346143,
   -0.06863713,
   -0.07418612,
   -0.02515379,
   0

In [9]:
list(relation2embedding.items())[:2]

[('term0',
  [-0.14911288,
   -0.065331765,
   -0.053362478,
   0.0021824413,
   0.13459179,
   0.010671005,
   -0.0076839332,
   0.02729784,
   0.007890213,
   -0.010737106,
   -0.072772,
   0.0041414,
   -0.027336225,
   0.008390477,
   0.015475275,
   -0.0043292707,
   0.1381746,
   -0.00086962467,
   0.02643781,
   0.041113816,
   -0.037475586,
   0.13188638,
   -0.052933894,
   0.0014266283,
   -0.0075507932,
   -0.18545148,
   0.008612064,
   -0.047192924,
   -0.0005759825,
   -0.0032967718,
   -0.1948991,
   -0.067344815,
   -0.0716464,
   0.15143411,
   0.06934708,
   0.009265065,
   -0.121763304,
   -0.06612246,
   0.19337,
   -0.03550226,
   0.11574635,
   -0.13020562,
   -0.011805415,
   -0.11933036,
   0.0033138557,
   0.11462386,
   -0.1382508,
   0.01456391,
   0.050730936,
   0.010925047,
   0.069670685,
   -0.18719822,
   0.1516529,
   0.0017688142,
   -0.20186466,
   0.034739643,
   0.13697033,
   0.00078189844,
   -0.12725213,
   -0.05117121,
   -0.01094034,
   0.1006

In [10]:
#get scoring-function value of a knowledge-graph triple
h1,r1,t1 = 'person0', 'term0', 'person1'
h2,r2,t2 = 'person2', 'term1', 'person3'
h1id,r1id,t1id = all_triples.entity_to_id[h1], all_triples.relation_to_id[r1], all_triples.entity_to_id[t1]
h2id,r2id,t2id = all_triples.entity_to_id[h2], all_triples.relation_to_id[r2], all_triples.entity_to_id[t2]
triples_to_be_scored = torch.LongTensor([[h1id,r1id,t1id],[h2id,r2id,t2id]])
result.model.cpu()
scores = result.model.score_hrt(triples_to_be_scored)
scores_numpy = scores.detach().numpy()
scores_numpy

array([[-20.105736],
       [-22.392626]], dtype=float32)